In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("Titanic-Dataset.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [4]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
#filling missing values of age column with median
df['Age'].fillna(df['Age'].median())

#filling column embarked with mode
df['Embarked'].fillna(df['Embarked'].mode()[0])

#droping cabin column coz too many missing values
df.drop('Cabin', axis=1, inplace=True)

feature engineering

In [7]:
df['familysize'] = df['SibSp'] + df['Parch'] + 1

df['AgeBin'] = pd.cut(df['Age'], bins=[0,12,20,40,60,80], labels=['Child', 'Teen', 'Adult', 'Middle-Age', 'Senoir'])

In [8]:
##Titles from Name

df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

#combine rare titles
df['Title']= df['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 
                                       'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
df['Title']=df['Title'].replace('Mlle', 'Miss')
df['Title']=df['Title'].replace('Ms', 'Miss')
df['Title']=df['Title'].replace('Mme', 'Mrs')

<>:3: SyntaxWarning: invalid escape sequence '\.'
<>:3: SyntaxWarning: invalid escape sequence '\.'
C:\Users\UJWAL\AppData\Local\Temp\ipykernel_13288\3422639364.py:3: SyntaxWarning: invalid escape sequence '\.'
  df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)


## Encode Categorical Variables


In [9]:
df_encoded = pd.get_dummies(df, columns=['Sex', 'Embarked', 'AgeBin', 'Title'],drop_first=True)

## select features and Target

In [13]:
features = ['Pclass', 'Fare', 'familysize'] + [col for col in df_encoded.columns if 'Sex_' in col or 'Embarked_' in col or 'AgeBin_' in col or 'Title_' in col]
X = df_encoded[features]
y = df_encoded['Survived']

## Train Logistic Regression

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_val)
print(confusion_matrix(y_val, y_pred))
print(classification_report(y_val, y_pred))

[[87 18]
 [18 56]]
              precision    recall  f1-score   support

           0       0.83      0.83      0.83       105
           1       0.76      0.76      0.76        74

    accuracy                           0.80       179
   macro avg       0.79      0.79      0.79       179
weighted avg       0.80      0.80      0.80       179



## Analyze Top Predictors

In [19]:
import numpy as np
importance = pd.DataFrame({'Feature':X.columns, 'Coefficient':model.coef_[0]})
importance['AbsCoeff'] = np.abs(importance['Coefficient'])
importance.sort_values(by='AbsCoeff', ascending=False,inplace=True)
print(importance[['Feature', 'Coefficient']])

              Feature  Coefficient
11           Title_Mr    -2.650816
13         Title_Rare    -1.721708
0              Pclass    -0.812998
3            Sex_male    -0.594793
8   AgeBin_Middle-Age    -0.529132
5          Embarked_S    -0.496133
12          Title_Mrs     0.481145
10         Title_Miss    -0.434503
2          familysize    -0.343803
9       AgeBin_Senoir    -0.304200
4          Embarked_Q    -0.269808
6         AgeBin_Teen    -0.125570
7        AgeBin_Adult     0.032905
1                Fare     0.003943


In [20]:
data_for_tableau=df[['Survived', 'Sex', 'Pclass', 'Age', 'Fare', 'familysize','Title','Embarked']]
data_for_tableau.to_csv('titanic_for_tableau.csv', index=False)